<a href="https://colab.research.google.com/github/Vivianflima/Projeto_ImersaoIA_MeuSimulado/blob/main/MeuSimulado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot para Criação de Provas com IA 🤖📝

## Objetivo

Este projeto utiliza o modelo de linguagem Google Gemini (`gemini-inst`) e inteligência artificial para auxiliar estudantes na criação de perguntas para provas e simulados com base em material de estudo em PDF e provas de exemplo.

## Funcionalidades

*   Extrai texto de arquivos PDF de material de estudo.
*   Analisa provas de exemplo (opcional) para entender o estilo e a dificuldade das perguntas.
*   Gera 5 perguntas com alternativas utilizando o modelo `gemini-inst`.
*   Oferece uma interface interativa para os usuários responderem às perguntas.
*   Fornece feedback com a nota do usuário e as respostas corretas.

## Tecnologias Utilizadas

*   Google Gemini (`gemini-inst`)
*   pymupdf
*   ipywidgets (ou outra biblioteca para interface de usuário)

## Instruções

1. **Forneça a URL do GitHub:** Insira a URL do repositório onde o PDF do material de estudo está armazenado.
2. **Prova de Exemplo (Opcional):** Forneça a URL de um PDF com uma prova de exemplo, se desejar.
3. **Gere as Perguntas:** O chatbot irá gerar 5 perguntas com alternativas.
4. **Responda às Perguntas:** Selecione a alternativa correta para cada pergunta.
5. **Veja o Feedback:** O chatbot apresentará sua nota e as respostas corretas.

### Observação Importante

Antes de executar o chatbot, certifique-se de inserir sua API key do Google no campo a seguir:

`GOOGLE_API_KEY="sua_api"`


# Instalação das Bibliotecas

Nesta célula, instalamos as bibliotecas necessárias para o projeto:

* google-generativeai: Para utilizar os modelos de linguagem do Google.
* PyPDF2: Para extrair texto de arquivos PDF.


In [86]:
!pip install -q -U google-generativeai
!pip install -U PyPDF2

In [87]:
pip install transformers

In [88]:
!pip install pymupdf

In [89]:
#Configurações iniciais
import google.generativeai as genai

GOOGLE_API_KEY="sua_api"
genai.configure(api_key=GOOGLE_API_KEY)

In [90]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [91]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [92]:
model = genai.GenerativeModel(model_name='models/chat-bison-001',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

# Extração de Texto de PDFs do GitHub 📄➡️💬

**Esta célula define uma função que extrai o texto de um arquivo PDF armazenado no GitHub.**
Ela ajusta automaticamente a URL para o formato correto e utiliza a biblioteca `PyMuPDF` para ler o PDF e retornar o texto extraído.

In [93]:
import requests
import fitz

def extrair_texto_pdf_github(url_github):
    """
    Extrai o texto de um arquivo PDF armazenado em um repositório do GitHub.
    Ajusta a URL automaticamente para o formato raw.

    Args:
        url_github: A URL do arquivo PDF no GitHub (blob ou raw).

    Returns:
        str: O texto extraído do PDF, ou uma mensagem de erro.
    """

    # Ajustar a URL para o formato raw
    if "/blob/" in url_github:
        url_github = url_github.replace("/blob/", "/raw/")

    try:
        response = requests.get(url_github)
        response.raise_for_status()

        doc = fitz.open(stream=response.content, filetype="pdf")
        texto = ""
        for pagina in doc:
            texto += pagina.get_text()

        return texto

    except requests.exceptions.RequestException as e:
        return f"Erro ao acessar o arquivo: {e}"

    except Exception as e:
        return f"Erro ao processar o arquivo PDF: {e}"

In [94]:
pip install ipywidgets

# Extração de Texto de PDFs da Prova de exemplo do GitHub 📄➡️💬

**Esta célula define uma função que extrai o formato da prova de exemplo de um arquivo PDF armazenado no GitHub.**

Esta célula contém uma função denominada `processar_prova_exemplo`, projetada para analisar uma prova de exemplo em formato PDF, fornecida por meio de uma URL do GitHub. A função extrai dados relevantes da prova, incluindo seu estilo, dificuldade e as perguntas com suas respectivas alternativas.

A função utiliza expressões regulares para identificar as perguntas de múltipla escolha no texto do PDF. Em seguida, determina o estilo e a dificuldade das perguntas, assumindo, para simplificação, que todas são de múltipla escolha e têm uma dificuldade média.

O resultado é retornado como um dicionário que contém informações sobre o estilo, dificuldade e as perguntas encontradas na prova, ou `None` se nenhuma prova for fornecida.


In [95]:
import re

def processar_prova_exemplo(url_prova):
    """
    Processa uma prova de exemplo em PDF e extrai informações relevantes.

    Args:
        url_prova: A URL do arquivo PDF da prova de exemplo no GitHub.

    Returns:
        dict: Um dicionário com informações sobre o estilo e a dificuldade das
              perguntas, ou None se nenhuma prova for fornecida.
    """

    if not url_prova:
        return None

    texto_prova = extrair_texto_pdf_github(url_prova)

    # Padrões de expressões regulares para identificar perguntas de múltipla escolha
    padrao_pergunta = r"^\d+\.\s*(.*?)\n(?:[A-Z]\.\s*(.*?)\n)+"
    padrao_alternativa = r"[A-Z]\.\s*(.*?)\n"

    # Extrair perguntas e alternativas
    perguntas = []
    for match in re.finditer(padrao_pergunta, texto_prova, re.MULTILINE):
        pergunta = match.group(1).strip()
        alternativas = re.findall(padrao_alternativa, match.group(0))
        perguntas.append({"pergunta": pergunta, "alternativas": alternativas})

    # Analisar o estilo e a dificuldade (simplificado para demonstração)
    estilo = "multipla_escolha"  # Assumindo que todas as perguntas são de múltipla escolha
    dificuldade = "medio"  # Assumindo dificuldade média para todas as perguntas

    return {"estilo": estilo, "dificuldade": dificuldade, "perguntas": perguntas}

# Extração de Texto de PDFs do GitHub 📄➡️💬

**Esta célula define uma função que extrai o texto de um arquivo PDF armazenado no GitHub.** Ela ajusta automaticamente a URL para o formato correto e utiliza a biblioteca `PyMuPDF` para ler o PDF e retornar o texto extraído.


In [119]:
import requests
import fitz  # Importe a biblioteca fitz

def extrair_assunto_pdf(url_github):
    """Extrai o assunto (título) do PDF usando fitz."""
    if "/blob/" in url_github:
        url_github = url_github.replace("/blob/", "/raw/")
    try:
        response = requests.get(url_github)
        response.raise_for_status()

        doc = fitz.open(stream=response.content, filetype="pdf")

        # Extrair o título dos metadados
        assunto = doc.metadata['title'] if 'title' in doc.metadata else 'Assunto não encontrado'
        return assunto

    except requests.exceptions.RequestException as e:
        return f"Erro ao acessar o arquivo: {e}"
    except Exception as e:
        return f"Erro ao processar o arquivo PDF: {e}"

In [97]:
!pip install spacy

In [98]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 29.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [99]:
!pip install google-generativeai

In [100]:
!pip install sentence_transformers

# Geração de Perguntas a partir de Texto de PDF 📄➡️❓

**Esta célula define uma função que gera perguntas com base no texto de um PDF.**
A função utiliza processamento de linguagem natural para identificar frases significativas no texto e criar perguntas a partir delas. As alternativas são geradas aleatoriamente, incluindo sinônimos e antônimos quando aplicável.

In [109]:
import nltk
import spacy
import random

def gerar_perguntas(texto_pdf, num_perguntas=5, prova_exemplo=None, doc=None):
    """
    Gera perguntas com base no texto do PDF.

    Args:
        texto_pdf: O texto extraído do PDF.
        num_perguntas: O número de perguntas a serem geradas.
        prova_exemplo: Um dicionário com informações sobre a prova de exemplo,
                       ou None se nenhuma prova for fornecida.
        doc: O objeto Doc do spaCy contendo o texto processado.

    Returns:
        list: Uma lista de dicionários, onde cada dicionário representa uma
              pergunta e contém as chaves "pergunta", "alternativas" e "resposta".
    """

    def gerar_alternativas(frase_chave):
        """
        Gera alternativas plausíveis para uma frase-chave.
        """
        alternativas = []

        # 1. Encontrar frases aleatórias do texto
        frases_aleatorias = [sent.text for sent in doc.sents if len(sent) > 8 and sent.text != frase_chave]
        while len(alternativas) < 3 and frases_aleatorias:
            alternativas.append(random.choice(frases_aleatorias))
            frases_aleatorias.remove(alternativas[-1])  # Evitar repetições

        # 2. Adicionar sinônimos e antônimos do WordNet (opcional)
        for token in nlp(frase_chave):
            if token.pos_ in ["NOUN", "VERB", "ADJ"]:
                sinonimos = []
                antonimos = []
                for synset in nltk.corpus.wordnet.synsets(token.text):
                    for lemma in synset.lemmas():
                        if lemma.name() != token.text:
                            sinonimos.append(lemma.name())
                        if lemma.antonyms():
                            antonimos.append(lemma.antonyms()[0].name())

                if sinonimos:
                    alternativas.append(random.choice(sinonimos))
                if antonimos:
                    alternativas.append(random.choice(antonimos))

        random.shuffle(alternativas)
        return alternativas[:3]  # Limitar a 3 alternativas

    nlp = spacy.load("pt_core_news_sm")  # Carregar modelo de português

    perguntas = []
    while len(perguntas) < num_perguntas:
        # Encontrar uma frase que expresse um conceito ou relação
        frases_candidatas = [sent.text for sent in doc.sents if len(sent) > 10]
        frase_chave = random.choice(frases_candidatas)

        alternativas = gerar_alternativas(frase_chave)
        resposta_correta = frase_chave

        perguntas.append({"pergunta": frase_chave, "alternativas": alternativas, "resposta": resposta_correta})

    return perguntas


In [114]:
import nltk
import spacy
import random
from sentence_transformers import SentenceTransformer, util
import google.generativeai as genai


nlp = spacy.load("pt_core_news_sm")
model_embedding = SentenceTransformer('all-mpnet-base-v2')

def gerar_perguntas(texto_pdf, prova_exemplo, doc, assunto_do_pdf, num_perguntas=5):
    """
    Gera perguntas de múltipla escolha relevantes com base no texto do PDF,
    na prova de exemplo e no assunto do PDF.

    Args:
        texto_pdf: O texto extraído do PDF.
        prova_exemplo: Um dicionário com informações da prova de exemplo.
        doc: O documento processado pelo spaCy.
        assunto_do_pdf: O assunto principal do PDF.
        num_perguntas: O número de perguntas a serem geradas.

    Returns:
        list: Uma lista de dicionários, onde cada dicionário representa uma pergunta
              com sua pergunta, alternativas e resposta correta.
    """

    def gerar_alternativas(frase_chave):
        """
        Gera alternativas plausíveis para uma frase-chave.
        """
        alternativas = []

        # 1. Encontrar frases aleatórias do texto:
        frases_aleatorias = [sent.text for sent in doc.sents if len(sent) > 8 and sent.text != frase_chave]
        while len(alternativas) < 3 and frases_aleatorias:
            alternativas.append(random.choice(frases_aleatorias))
            frases_aleatorias.remove(alternativas[-1])  # Evitar repetições

        # 2. Adicionar sinônimos e antônimos do WordNet (opcional):
        # ... (Implemente se desejar)

        random.shuffle(alternativas)
        return alternativas[:3]  # Limitar a 3 alternativas

    perguntas = []

    # 1. Definir o estilo e dificuldade:
    estilo = prova_exemplo.get('estilo', 'multipla_escolha')
    dificuldade = prova_exemplo.get('dificuldade', 'medio')

    # 2. Dividir o texto em seções:
    tamanho_secao = 500
    secoes = [texto_pdf[i:i + tamanho_secao] for i in range(0, len(texto_pdf), tamanho_secao)]

    # 3. Gerar perguntas:
    for _ in range(num_perguntas):
        # 3.1 Encontrar a seção mais relevante:
        secao_relevante, maior_similaridade = None, 0

        if prova_exemplo:
          exemplo_pergunta = prova_exemplo['perguntas'][0]['pergunta']
          embedding_exemplo = model_embedding.encode(exemplo_pergunta, convert_to_tensor=True)

          for secao in secoes:
              doc_secao = nlp(secao)
              embeddings_secao = model_embedding.encode(doc_secao.sents, convert_to_tensor=True)
              similaridade = util.pytorch_cos_sim(embedding_exemplo, embeddings_secao).max()
              if similaridade > maior_similaridade:
                  maior_similaridade = similaridade
                  secao_relevante = secao
        else:
          secao_relevante = random.choice(secoes)

        # 3.2 Criar o prompt para o PaLM 2:
        prompt = f"""
        Você é um professor especialista em {assunto_do_pdf}.
        Crie uma questão de múltipla escolha com 4 alternativas,
        com nível de dificuldade {dificuldade},
        com base no seguinte texto:

        {secao_relevante}

        Formato da resposta:
        Pergunta: ...
        A. ...
        B. ...
        C. ...
        D. ...
        Resposta correta: ...
        """

        # 3.3 Gerar a pergunta com o PaLM 2:
        resposta_palm = model.generate_text(prompt=prompt).result

        # 3.4 Processar a resposta do PaLM 2:
        match = re.search(
            r"Pergunta: (.*?)\nA\. (.*?)\nB\. (.*?)\nC\. (.*?)\nD\. (.*?)\nResposta correta: ([A-D])",
            resposta_palm,
            re.DOTALL,
        )

        if match:
            pergunta = match.group(1).strip()
            alternativas = [match.group(i).strip() for i in range(2, 6)]
            resposta_correta = match.group(6)
            perguntas.append({
                'pergunta': pergunta,
                'alternativas': alternativas,
                'resposta_correta': resposta_correta
            })
        else:
            print(f"Não foi possível analisar a resposta do PaLM: {resposta_palm}")

    return perguntas

In [102]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [103]:
!python -m spacy download pt_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 7.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Interface de Geração de Perguntas a partir de PDFs 📄❓

**Esta célula cria uma interface interativa para gerar perguntas com base no texto de um PDF hospedado no GitHub.**

## Widgets Utilizados

* **URL do PDF do seu Repositório no GitHub:** Campo de entrada para a URL do PDF.
* **URL da Prova de Exemplo do seu Repositório do GitHub (Opcional):** Campo de entrada para a URL da prova de exemplo, se houver.
* **Gerar Perguntas:** Botão para iniciar o processo de geração de perguntas.
* **Output:** Área de saída para exibir as perguntas, opções de resposta e feedback do usuário.

## Funcionalidades

1. **Extrair Texto do PDF:** A função `extrair_texto_pdf_github` é usada para extrair o texto do PDF fornecido.
2. **Processar Prova de Exemplo (Opcional):** A função `processar_prova_exemplo` é usada para processar a prova de exemplo, se fornecida.
3. **Gerar Perguntas:** A função `gerar_perguntas` é usada para gerar as perguntas com base no texto do PDF e na prova de exemplo, se disponível.
4. **Responder às Perguntas:** Os usuários podem selecionar as alternativas para cada pergunta usando RadioButtons.
5. **Enviar Respostas:** Ao clicar no botão "Enviar Respostas", o sistema calcula a nota do usuário e exibe as respostas corretas.

*Nota: Certifique-se de fornecer as URLs corretas e aguarde o processamento após clicar em "Gerar Perguntas".*


In [118]:
from ipywidgets import Text, Button, Output, Layout, HTML, RadioButtons

# Criar os widgets
input_url_pdf = Text(description="URL do PDF do seu repositório no Github:",
                     layout=Layout(width="700px"))
input_url_prova = Text(
    description="URL da Prova de exemplo do seu repositório do Github (opcional):",
    layout=Layout(width="700px")  # Ajustar a largura do campo de entrada
)

# CSS personalizado para o título
estilo_titulo = """
<style>
.widget-label {
    min-width: 400px;  /* Ajustar a largura mínima do título */
}
</style>
"""
# Criar um widget HTML para injetar o CSS
widget_css = HTML(estilo_titulo)

botao_gerar = Button(description="Gerar Perguntas")
output_area = Output()

# Exibir os widgets
display(widget_css, input_url_pdf, input_url_prova, botao_gerar, output_area)

def on_button_clicked(b):
    url_pdf = input_url_pdf.value
    url_prova = input_url_prova.value

    texto_pdf = extrair_texto_pdf_github(url_pdf)
    prova_exemplo = processar_prova_exemplo(url_prova)

    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(texto_pdf)

    with output_area:
        output_area.clear_output()
        print("Carregando...")

    perguntas = gerar_perguntas(texto_pdf, prova_exemplo=prova_exemplo, doc=doc, assunto_do_pdf=assunto_do_pdf)

    respostas_usuario = {}  # Dicionário para armazenar as respostas

    # Limpar a mensagem de carregamento antes de exibir as perguntas
    with output_area:
        output_area.clear_output()

        # Exibir as perguntas e RadioButtons
        for i, pergunta in enumerate(perguntas):
            print(f"{i+1}. {pergunta['pergunta']}")
            alternativas_widget = RadioButtons(
                options=pergunta['alternativas'],
                description='',
                disabled=False
            )
            display(alternativas_widget)

            def on_alternativa_change(change):
                respostas_usuario[i] = change['new']

            alternativas_widget.observe(on_alternativa_change, names='value')

        # Botão para enviar as respostas
        botao_enviar = Button(description="Enviar Respostas")
        display(botao_enviar)

        def on_enviar_click(b):
            # Processar as respostas do usuário
            corretas = 0
            for i, pergunta in enumerate(perguntas):
                if respostas_usuario.get(i) == pergunta['resposta']:
                    corretas += 1

            nota = (corretas / len(perguntas)) * 10
            print(f"Você acertou {corretas} de {len(perguntas)} perguntas.")
            print(f"Sua nota: {nota:.2f}")

            # Exibir as respostas corretas
            print("\nRespostas corretas:")
            for i, pergunta in enumerate(perguntas):
                print(f"{i+1}. {pergunta['pergunta']}")
                print(f"   Resposta correta: {pergunta['resposta']}")

        botao_enviar.on_click(on_enviar_click)

# Conectar a função ao botão
botao_gerar.on_click(on_button_clicked)

HTML(value='\n<style>\n.widget-label {\n    min-width: 400px;  /* Ajustar a largura mínima do título */\n}\n</…

Text(value='', description='URL do PDF do seu repositório no Github:', layout=Layout(width='700px'))

Text(value='', description='URL da Prova de exemplo do seu repositório do Github (opcional):', layout=Layout(w…

Button(description='Gerar Perguntas', style=ButtonStyle())

Output()